In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import gc
import sys
sys.path.append('../')

from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)
from IPython.display import HTML, display
from sklearn import metrics
import traceback 
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

import featuretools as ft

MERED = "merged"
PATH = "../../../data/home-credit-default-risk/"
#PATH = "../../data/home-credit-default-risk/"

/home/max/anaconda3/envs/fastai-cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/max/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [6]:
ls {PATH}

application_test.csv                merged/
application_train.csv               models/
bureau_balance.csv                  POS_CASH_balance.csv
bureau.csv                          previous_application.csv
credit_card_balance.csv             sample_submission.csv
HomeCredit_columns_description.csv  tmp/
installments_payments.csv


In [132]:
n_sample = 1000

In [59]:
def delete_by_nan(table, ratio=0.5):
    null_ratio = table.isnull().sum().sort_values(ascending=False) / len(table)
    filtered_by_null = null_ratio[null_ratio > ratio]
    
    table.drop(columns=filtered_by_null.index, axis=1, inplace=True)

In [60]:
def load_sample(file_name, sk_id_currs=None, nrows=None):
    table = pd.read_csv(f'{PATH}{file_name}.csv', low_memory=False, nrows=nrows)
    if sk_id_currs is not None:
        table = table[table.SK_ID_CURR.isin(sk_id_currs)]    
    
    col_num = len(table.columns)
    delete_by_nan(table)
    
    print(file_name, len(table), col_num, len(table.columns))
    return table

In [118]:
def add_table_to_app(app, table, table_name):
    entities = {
        'app' : (app, "SK_ID_CURR"),
        table_name : (table, 'index'),
    }
    relationships = [
        ('app', 'SK_ID_CURR', table_name, 'index')
    ]
    
    return add_table_to_app_by_entities(entities=entities, relationships=relationships, table_name=table_name)

In [119]:
def add_table_to_app_by_entities(entities, relationships, table_name):
    feature_matrix, feature_defs = ft.dfs(
        entities=entities,
        relationships = relationships,
        target_entity="app",
        agg_primitives=["mean", "sum", "mode"],
        ignore_variables={table_name: ['SK_ID_PREV', 'SK_ID_CURR']}
    )
    
    os.makedirs(f'{PATH}tmp', exist_ok=True)
    feature_matrix = feature_matrix.reset_index()
    
    feature_matrix.to_feather(f'{PATH}tmp/app-table')        
    
    return feature_matrix, feature_defs

In [162]:
train = load_sample('application_train',  nrows=n_sample)

application_train 1000 122 92


In [163]:
test = load_sample('application_test',  nrows=n_sample)

application_test 1000 121 92


In [164]:
train['is_test'] = 0
df_t_t = pd.concat([train, test])
df_t_t.is_test.fillna(1, inplace=True)

/home/max/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [165]:
del train, test
gc.collect()

1308

In [166]:
bureau = load_sample('bureau', df_t_t.SK_ID_CURR)

bureau 9665 17 15


In [167]:
file_names = ['bureau_balance']
tables = [pd.read_csv(f'{PATH}{file_name}.csv',low_memory=False) for file_name in file_names]
bureau_balance = tables[0][tables[0].SK_ID_BUREAU.isin(bureau.SK_ID_BUREAU)]
bureau_balance = bureau_balance.reset_index()

In [168]:
entities = {
    'app' : (df_t_t, "SK_ID_CURR"),
    'bureau' : (bureau, "SK_ID_BUREAU"),
    'bureau_balance' : (bureau_balance, 'index')
}

relationships = [
    ('app', 'SK_ID_CURR', 'bureau', 'SK_ID_CURR'),
    ('bureau', 'SK_ID_BUREAU', 'bureau_balance', 'SK_ID_BUREAU')
]

In [169]:
df_t_t, _ = add_table_to_app_by_entities(entities, relationships, 'bureau')

In [170]:
del bureau, bureau_balance, tables
gc.collect()

802

In [171]:
file_names = ['previous_application', 'POS_CASH_balance', 'credit_card_balance', 'installments_payments']
for file_name in file_names:
    table = load_sample(file_name, df_t_t.SK_ID_CURR)
    table = table.reset_index()
    df_t_t, feature_defs = add_table_to_app(df_t_t, table, file_name)

previous_application 9948 37 33
POS_CASH_balance 58981 8 8
credit_card_balance 22632 23 23
installments_payments 79690 8 8


In [215]:
df_all = pd.read_feather(f'{PATH}tmp/app-table')

/home/max/.local/lib/python3.6/site-packages/pandas/io/feather_format.py:112: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_dataframe(path, nthreads=nthreads)


In [203]:
df_all.head()

,SK_ID_CURR,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,...,MEAN(installments_payments.DAYS_INSTALMENT),MEAN(installments_payments.DAYS_ENTRY_PAYMENT),MEAN(installments_payments.AMT_INSTALMENT),MEAN(installments_payments.AMT_PAYMENT),SUM(installments_payments.NUM_INSTALMENT_VERSION),SUM(installments_payments.NUM_INSTALMENT_NUMBER),SUM(installments_payments.DAYS_INSTALMENT),SUM(installments_payments.DAYS_ENTRY_PAYMENT),SUM(installments_payments.AMT_INSTALMENT),SUM(installments_payments.AMT_PAYMENT)
0,100001,20560.5,568800.0,450000.0,135000.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100002,24700.5,406597.5,351000.0,202500.0,0.0,0.0,0.0,0.0,0.0,...,-778.0,-789.0,447329.61,447329.61,2.0,6.0,-778.0,-789.0,447329.61,447329.61
2,100003,35698.5,1293502.5,1129500.0,270000.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100004,6750.0,135000.0,135000.0,67500.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100005,17370.0,222768.0,180000.0,99000.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [204]:
 with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000):
        display(DataFrameSummary(df_all).summary())

,SK_ID_CURR,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,APARTMENTS_AVG,APARTMENTS_MEDI,APARTMENTS_MODE,CNT_CHILDREN,CNT_FAM_MEMBERS,CODE_GENDER,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_ID_PUBLISH,DAYS_LAST_PHONE_CHANGE,DAYS_REGISTRATION,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,EMERGENCYSTATE_MODE,ENTRANCES_AVG,ENTRANCES_MEDI,ENTRANCES_MODE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,FLAG_CONT_MOBILE,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_2,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_EMAIL,FLAG_EMP_PHONE,FLAG_MOBIL,FLAG_OWN_CAR,FLAG_OWN_REALTY,FLAG_PHONE,FLAG_WORK_PHONE,FLOORSMAX_AVG,FLOORSMAX_MEDI,FLOORSMAX_MODE,HOUR_APPR_PROCESS_START,HOUSETYPE_MODE,LIVE_CITY_NOT_WORK_CITY,LIVE_REGION_NOT_WORK_REGION,LIVINGAREA_AVG,LIVINGAREA_MEDI,LIVINGAREA_MODE,NAME_CONTRACT_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,NAME_INCOME_TYPE,NAME_TYPE_SUITE,OBS_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,OCCUPATION_TYPE,ORGANIZATION_TYPE,REGION_POPULATION_RELATIVE,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,TARGET,TOTALAREA_MODE,WALLSMATERIAL_MODE,WEEKDAY_APPR_PROCESS_START,YEARS_BEGINEXPLUATATION_AVG,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BEGINEXPLUATATION_MODE,is_test,MEAN(bureau.DAYS_CREDIT),MEAN(bureau.CREDIT_DAY_OVERDUE),MEAN(bureau.DAYS_CREDIT_ENDDATE),MEAN(bureau.DAYS_ENDDATE_FACT),MEAN(bureau.CNT_CREDIT_PROLONG),MEAN(bureau.AMT_CREDIT_SUM),MEAN(bureau.AMT_CREDIT_SUM_DEBT),MEAN(bureau.AMT_CREDIT_SUM_LIMIT),MEAN(bureau.AMT_CREDIT_SUM_OVERDUE),MEAN(bureau.DAYS_CREDIT_UPDATE),SUM(bureau.DAYS_CREDIT),SUM(bureau.CREDIT_DAY_OVERDUE),SUM(bureau.DAYS_CREDIT_ENDDATE),SUM(bureau.DAYS_ENDDATE_FACT),SUM(bureau.CNT_CREDIT_PROLONG),SUM(bureau.AMT_CREDIT_SUM),SUM(bureau.AMT_CREDIT_SUM_DEBT),SUM(bureau.AMT_CREDIT_SUM_LIMIT),SUM(bureau.AMT_CREDIT_SUM_OVERDUE),SUM(bureau.DAYS_CREDIT_UPDATE),MODE(bureau.CREDIT_ACTIVE),MODE(bureau.CREDIT_CURRENCY),MODE(bureau.CREDIT_TYPE),MEAN(bureau_balance.MONTHS_BALANCE),SUM(bureau_balance.MONTHS_BALANCE),MODE(bureau_balance.STATUS),MEAN(bureau.MEAN(bureau_balance.MONTHS_BALANCE)),MEAN(bureau.SUM(bureau_balance.MONTHS_BALANCE)),SUM(bureau.MEAN(bureau_balance.MONTHS_BALANCE)),MODE(bureau.MODE(bureau_balance.STATUS)),MEAN(previous_application.AMT_ANNUITY),MEAN(previous_application.AMT_APPLICATION),MEAN(previous_application.AMT_CREDIT),MEAN(previous_application.AMT_GOODS_PRICE),MEAN(previous_application.HOUR_APPR_PROCESS_START),MEAN(previous_application.NFLAG_LAST_APPL_IN_DAY),MEAN(previous_application.DAYS_DECISION),MEAN(previous_application.SELLERPLACE_AREA),MEAN(previous_application.CNT_PAYMENT),MEAN(previous_application.DAYS_FIRST_DRAWING),MEAN(previous_application.DAYS_FIRST_DUE),MEAN(previous_application.DAYS_LAST_DUE_1ST_VERSION),MEAN(previous_application.DAYS_LAST_DUE),MEAN(previous_application.DAYS_TERMINATION),MEAN(previous_application.NFLAG_INSURED_ON_APPROVAL),SUM(previous_application.AMT_ANNUITY),SUM(previous_application.AMT_APPLICATION),SUM(previous_application.AMT_CREDIT),SUM(previous_application.AMT_GOODS_PRICE),SUM(previous_application.HOUR_APPR_PROCESS_START),SUM(previous_application.NFLAG_LAST_APPL_IN_DAY),SUM(previous_application.DAYS_DECISION),SUM(previous_application.SELLERPLACE_AREA),SUM(previous_application.CNT_PAYMENT),SUM(previous_application.DAYS_FIRST_DRAWING),SUM(previous_application.DAYS_FIRST_DUE),SUM(previous_application.DAYS_LAST_DUE_1ST_VERSION),SUM(previous_application.DAYS_LAST_DUE),SUM(previous_application.DAYS_TERMINATION),SUM(previous_application.NFLAG_INSURED_O

In [176]:
delete_by_nan(df_all)

In [177]:
[print(c) for c in df_all.columns]

SK_ID_CURR
AMT_ANNUITY
AMT_CREDIT
AMT_GOODS_PRICE
AMT_INCOME_TOTAL
AMT_REQ_CREDIT_BUREAU_DAY
AMT_REQ_CREDIT_BUREAU_HOUR
AMT_REQ_CREDIT_BUREAU_MON
AMT_REQ_CREDIT_BUREAU_QRT
AMT_REQ_CREDIT_BUREAU_WEEK
AMT_REQ_CREDIT_BUREAU_YEAR
APARTMENTS_AVG
APARTMENTS_MEDI
APARTMENTS_MODE
CNT_CHILDREN
CNT_FAM_MEMBERS
CODE_GENDER
DAYS_BIRTH
DAYS_EMPLOYED
DAYS_ID_PUBLISH
DAYS_LAST_PHONE_CHANGE
DAYS_REGISTRATION
DEF_30_CNT_SOCIAL_CIRCLE
DEF_60_CNT_SOCIAL_CIRCLE
EMERGENCYSTATE_MODE
ENTRANCES_AVG
ENTRANCES_MEDI
ENTRANCES_MODE
EXT_SOURCE_2
EXT_SOURCE_3
FLAG_CONT_MOBILE
FLAG_DOCUMENT_10
FLAG_DOCUMENT_11
FLAG_DOCUMENT_12
FLAG_DOCUMENT_13
FLAG_DOCUMENT_14
FLAG_DOCUMENT_15
FLAG_DOCUMENT_16
FLAG_DOCUMENT_17
FLAG_DOCUMENT_18
FLAG_DOCUMENT_19
FLAG_DOCUMENT_2
FLAG_DOCUMENT_20
FLAG_DOCUMENT_21
FLAG_DOCUMENT_3
FLAG_DOCUMENT_4
FLAG_DOCUMENT_5
FLAG_DOCUMENT_6
FLAG_DOCUMENT_7
FLAG_DOCUMENT_8
FLAG_DOCUMENT_9
FLAG_EMAIL
FLAG_EMP_PHONE
FLAG_MOBIL
FLAG_OWN_CAR
FLAG_OWN_REALTY
FLAG_PHONE
FLAG_WORK_PHONE
FLOORSMAX_AVG
FLOORSMA

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [205]:
train = df_all[df_all.is_test == 0]
train.drop(['is_test'], axis=1)
train = train.reset_index()
train.to_feather(f'{PATH}tmp/train_all_tables')

In [216]:
test = df_all[df_all.is_test == 1]
test.drop(['is_test'], axis=1)
test = test.reset_index()
test.to_feather(f'{PATH}tmp/test_all_tables')

In [219]:
del df_all, train, test
gc.collect()

NameError: name 'df_all' is not defined

In [207]:
train = pd.read_feather(f'{PATH}tmp/train_all_tables')

/home/max/.local/lib/python3.6/site-packages/pandas/io/feather_format.py:112: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_dataframe(path, nthreads=nthreads)


In [208]:
 with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000):
        display(DataFrameSummary(train).summary())

,index,SK_ID_CURR,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,APARTMENTS_AVG,APARTMENTS_MEDI,APARTMENTS_MODE,CNT_CHILDREN,CNT_FAM_MEMBERS,CODE_GENDER,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_ID_PUBLISH,DAYS_LAST_PHONE_CHANGE,DAYS_REGISTRATION,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,EMERGENCYSTATE_MODE,ENTRANCES_AVG,ENTRANCES_MEDI,ENTRANCES_MODE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,FLAG_CONT_MOBILE,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_2,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_EMAIL,FLAG_EMP_PHONE,FLAG_MOBIL,FLAG_OWN_CAR,FLAG_OWN_REALTY,FLAG_PHONE,FLAG_WORK_PHONE,FLOORSMAX_AVG,FLOORSMAX_MEDI,FLOORSMAX_MODE,HOUR_APPR_PROCESS_START,HOUSETYPE_MODE,LIVE_CITY_NOT_WORK_CITY,LIVE_REGION_NOT_WORK_REGION,LIVINGAREA_AVG,LIVINGAREA_MEDI,LIVINGAREA_MODE,NAME_CONTRACT_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,NAME_INCOME_TYPE,NAME_TYPE_SUITE,OBS_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,OCCUPATION_TYPE,ORGANIZATION_TYPE,REGION_POPULATION_RELATIVE,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,TARGET,TOTALAREA_MODE,WALLSMATERIAL_MODE,WEEKDAY_APPR_PROCESS_START,YEARS_BEGINEXPLUATATION_AVG,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BEGINEXPLUATATION_MODE,is_test,MEAN(bureau.DAYS_CREDIT),MEAN(bureau.CREDIT_DAY_OVERDUE),MEAN(bureau.DAYS_CREDIT_ENDDATE),MEAN(bureau.DAYS_ENDDATE_FACT),MEAN(bureau.CNT_CREDIT_PROLONG),MEAN(bureau.AMT_CREDIT_SUM),MEAN(bureau.AMT_CREDIT_SUM_DEBT),MEAN(bureau.AMT_CREDIT_SUM_LIMIT),MEAN(bureau.AMT_CREDIT_SUM_OVERDUE),MEAN(bureau.DAYS_CREDIT_UPDATE),SUM(bureau.DAYS_CREDIT),SUM(bureau.CREDIT_DAY_OVERDUE),SUM(bureau.DAYS_CREDIT_ENDDATE),SUM(bureau.DAYS_ENDDATE_FACT),SUM(bureau.CNT_CREDIT_PROLONG),SUM(bureau.AMT_CREDIT_SUM),SUM(bureau.AMT_CREDIT_SUM_DEBT),SUM(bureau.AMT_CREDIT_SUM_LIMIT),SUM(bureau.AMT_CREDIT_SUM_OVERDUE),SUM(bureau.DAYS_CREDIT_UPDATE),MODE(bureau.CREDIT_ACTIVE),MODE(bureau.CREDIT_CURRENCY),MODE(bureau.CREDIT_TYPE),MEAN(bureau_balance.MONTHS_BALANCE),SUM(bureau_balance.MONTHS_BALANCE),MODE(bureau_balance.STATUS),MEAN(bureau.MEAN(bureau_balance.MONTHS_BALANCE)),MEAN(bureau.SUM(bureau_balance.MONTHS_BALANCE)),SUM(bureau.MEAN(bureau_balance.MONTHS_BALANCE)),MODE(bureau.MODE(bureau_balance.STATUS)),MEAN(previous_application.AMT_ANNUITY),MEAN(previous_application.AMT_APPLICATION),MEAN(previous_application.AMT_CREDIT),MEAN(previous_application.AMT_GOODS_PRICE),MEAN(previous_application.HOUR_APPR_PROCESS_START),MEAN(previous_application.NFLAG_LAST_APPL_IN_DAY),MEAN(previous_application.DAYS_DECISION),MEAN(previous_application.SELLERPLACE_AREA),MEAN(previous_application.CNT_PAYMENT),MEAN(previous_application.DAYS_FIRST_DRAWING),MEAN(previous_application.DAYS_FIRST_DUE),MEAN(previous_application.DAYS_LAST_DUE_1ST_VERSION),MEAN(previous_application.DAYS_LAST_DUE),MEAN(previous_application.DAYS_TERMINATION),MEAN(previous_application.NFLAG_INSURED_ON_APPROVAL),SUM(previous_application.AMT_ANNUITY),SUM(previous_application.AMT_APPLICATION),SUM(previous_application.AMT_CREDIT),SUM(previous_application.AMT_GOODS_PRICE),SUM(previous_application.HOUR_APPR_PROCESS_START),SUM(previous_application.NFLAG_LAST_APPL_IN_DAY),SUM(previous_application.DAYS_DECISION),SUM(previous_application.SELLERPLACE_AREA),SUM(previous_application.CNT_PAYMENT),SUM(previous_application.DAYS_FIRST_DRAWING),SUM(previous_application.DAYS_FIRST_DUE),SUM(previous_application.DAYS_LAST_DUE_1ST_VERSION),SUM(previous_application.DAYS_LAST_DUE),SUM(previous_application.DAYS_TERMINATION),SUM(previous_application.NFLAG_INS

In [220]:
test = pd.read_feather(f'{PATH}tmp/test_all_tables')

/home/max/.local/lib/python3.6/site-packages/pandas/io/feather_format.py:112: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_dataframe(path, nthreads=nthreads)


In [221]:
with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000):
    display(DataFrameSummary(test).summary())

,index,SK_ID_CURR,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,APARTMENTS_AVG,APARTMENTS_MEDI,APARTMENTS_MODE,CNT_CHILDREN,CNT_FAM_MEMBERS,CODE_GENDER,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_ID_PUBLISH,DAYS_LAST_PHONE_CHANGE,DAYS_REGISTRATION,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,EMERGENCYSTATE_MODE,ENTRANCES_AVG,ENTRANCES_MEDI,ENTRANCES_MODE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,FLAG_CONT_MOBILE,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_2,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_EMAIL,FLAG_EMP_PHONE,FLAG_MOBIL,FLAG_OWN_CAR,FLAG_OWN_REALTY,FLAG_PHONE,FLAG_WORK_PHONE,FLOORSMAX_AVG,FLOORSMAX_MEDI,FLOORSMAX_MODE,HOUR_APPR_PROCESS_START,HOUSETYPE_MODE,LIVE_CITY_NOT_WORK_CITY,LIVE_REGION_NOT_WORK_REGION,LIVINGAREA_AVG,LIVINGAREA_MEDI,LIVINGAREA_MODE,NAME_CONTRACT_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,NAME_INCOME_TYPE,NAME_TYPE_SUITE,OBS_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,OCCUPATION_TYPE,ORGANIZATION_TYPE,REGION_POPULATION_RELATIVE,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,TARGET,TOTALAREA_MODE,WALLSMATERIAL_MODE,WEEKDAY_APPR_PROCESS_START,YEARS_BEGINEXPLUATATION_AVG,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BEGINEXPLUATATION_MODE,is_test,MEAN(bureau.DAYS_CREDIT),MEAN(bureau.CREDIT_DAY_OVERDUE),MEAN(bureau.DAYS_CREDIT_ENDDATE),MEAN(bureau.DAYS_ENDDATE_FACT),MEAN(bureau.CNT_CREDIT_PROLONG),MEAN(bureau.AMT_CREDIT_SUM),MEAN(bureau.AMT_CREDIT_SUM_DEBT),MEAN(bureau.AMT_CREDIT_SUM_LIMIT),MEAN(bureau.AMT_CREDIT_SUM_OVERDUE),MEAN(bureau.DAYS_CREDIT_UPDATE),SUM(bureau.DAYS_CREDIT),SUM(bureau.CREDIT_DAY_OVERDUE),SUM(bureau.DAYS_CREDIT_ENDDATE),SUM(bureau.DAYS_ENDDATE_FACT),SUM(bureau.CNT_CREDIT_PROLONG),SUM(bureau.AMT_CREDIT_SUM),SUM(bureau.AMT_CREDIT_SUM_DEBT),SUM(bureau.AMT_CREDIT_SUM_LIMIT),SUM(bureau.AMT_CREDIT_SUM_OVERDUE),SUM(bureau.DAYS_CREDIT_UPDATE),MODE(bureau.CREDIT_ACTIVE),MODE(bureau.CREDIT_CURRENCY),MODE(bureau.CREDIT_TYPE),MEAN(bureau_balance.MONTHS_BALANCE),SUM(bureau_balance.MONTHS_BALANCE),MODE(bureau_balance.STATUS),MEAN(bureau.MEAN(bureau_balance.MONTHS_BALANCE)),MEAN(bureau.SUM(bureau_balance.MONTHS_BALANCE)),SUM(bureau.MEAN(bureau_balance.MONTHS_BALANCE)),MODE(bureau.MODE(bureau_balance.STATUS)),MEAN(previous_application.AMT_ANNUITY),MEAN(previous_application.AMT_APPLICATION),MEAN(previous_application.AMT_CREDIT),MEAN(previous_application.AMT_GOODS_PRICE),MEAN(previous_application.HOUR_APPR_PROCESS_START),MEAN(previous_application.NFLAG_LAST_APPL_IN_DAY),MEAN(previous_application.DAYS_DECISION),MEAN(previous_application.SELLERPLACE_AREA),MEAN(previous_application.CNT_PAYMENT),MEAN(previous_application.DAYS_FIRST_DRAWING),MEAN(previous_application.DAYS_FIRST_DUE),MEAN(previous_application.DAYS_LAST_DUE_1ST_VERSION),MEAN(previous_application.DAYS_LAST_DUE),MEAN(previous_application.DAYS_TERMINATION),MEAN(previous_application.NFLAG_INSURED_ON_APPROVAL),SUM(previous_application.AMT_ANNUITY),SUM(previous_application.AMT_APPLICATION),SUM(previous_application.AMT_CREDIT),SUM(previous_application.AMT_GOODS_PRICE),SUM(previous_application.HOUR_APPR_PROCESS_START),SUM(previous_application.NFLAG_LAST_APPL_IN_DAY),SUM(previous_application.DAYS_DECISION),SUM(previous_application.SELLERPLACE_AREA),SUM(previous_application.CNT_PAYMENT),SUM(previous_application.DAYS_FIRST_DRAWING),SUM(previous_application.DAYS_FIRST_DUE),SUM(previous_application.DAYS_LAST_DUE_1ST_VERSION),SUM(previous_application.DAYS_LAST_DUE),SUM(previous_application.DAYS_TERMINATION),SUM(previous_application.NFLAG_INS